In [ ]:
import cv2
import numpy as np
import pandas as pd

# Misal kita punya sebuah frame dari OpenCV
frame = cv2.imread(r"C:\xampp\htdocs\VISUALAI\website-django\inspection\src\core\img1.jpg")  # Contoh, ganti path gambar
h, w, c = frame.shape  # c biasanya 3 (B,G,R)

# Buat grid koordinat x, y
# np.indices menghasilkan matriks 2D, shape (2, height, width)
coords = np.indices((h, w))
# coords[0] berisi kumpulan y, coords[1] berisi kumpulan x
y_coords = coords[0].flatten()
x_coords = coords[1].flatten()

# Flatten frame (H*W x 3)
pixels = frame.reshape(-1, 3)
# pixels[:, 0] = B, pixels[:, 1] = G, pixels[:, 2] = R

# Buat DataFrame
df = pd.DataFrame({
    "y": y_coords,
    "x": x_coords,
    "B": pixels[:, 0],
    "G": pixels[:, 1],
    "R": pixels[:, 2]
})

print(df.head(10))
print("Shape DataFrame:", df.shape)


In [ ]:
import os
import shutil


def is_image_file(filename):
    # Definisikan ekstensi gambar yang valid
    valid_ext = [".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff"]
    # Cek berdasarkan ekstensi lower-case
    ext = os.path.splitext(filename)[1].lower()
    return ext in valid_ext


def main():
    # Folder sumber (rekursif ditelusuri)
    root_path = r"C:\xampp\htdocs\VISUALAI\website-django\static\images\datasets\Jahitan"

    # Folder tujuan menampung subfolder-subfolder hasil pembagian
    # Misal: bikin folder "Jahitan_Grouped" di satu level di atas root_path
    # atau di mana pun Anda inginkan. Di contoh ini kita buat di level yang sama:
    parent_dir = os.path.dirname(root_path)
    dst_folder = os.path.join(parent_dir, "Jahitan_Grouped")
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)

    # Kumpulkan semua file gambar
    all_images = []
    for current_dir, dirs, files in os.walk(root_path):
        for f in files:
            if is_image_file(f):
                full_path = os.path.join(current_dir, f)
                all_images.append(full_path)

    total_images = len(all_images)
    print(f"Total file gambar ditemukan: {total_images}")

    # Bagi file menjadi subfolder, masing-masing 100 file
    batch_size = 100
    folder_index = 1

    # Urutkan jika ingin agar urutan penyalinan lebih terkontrol
    # (misal berdasarkan nama / path)
    all_images.sort()

    for i in range(0, total_images, batch_size):
        # Ambil slice gambar
        batch_files = all_images[i : i + batch_size]

        # Buat nama folder, ex: 01, 02, 03 ...
        subfolder_name = str(folder_index).zfill(2)
        subfolder_path = os.path.join(dst_folder, subfolder_name)

        if not os.path.exists(subfolder_path):
            os.makedirs(subfolder_path)

        # Salin (atau pindah) file ke subfolder ini
        for img_path in batch_files:
            fname = os.path.basename(img_path)
            dst_path = os.path.join(subfolder_path, fname)
            shutil.copy2(img_path, dst_path)

        print(f"Folder {subfolder_name} --> {len(batch_files)} file")
        folder_index += 1

    print("Proses selesai!")


if __name__ == "__main__":
    main()